In [18]:
# e = 65536
# e*d % phi(n) = 1
# e*e^(phi(phi(n))-1) = 1 % phi(n)
# d = e^(phi(phi(n))-1)
# ref: https://doc.sagemath.org/html/en/thematic_tutorials/numtheory_rsa.html
class RSA:
    def __init__(self, N):
        #p1 = random_prime(N)
        #p2 = random_prime(N)
        p1 = random_prime(N,1,N/2)
        p2 = random_prime(N,1,N/2)
        n = p1*p2
        phi = (p1-1)*(p2-1)
        e = Integers().random_element(phi)
        while gcd(e, phi)!=1:
            e = Integers().random_element(phi)
        self.e = e
        bezout = xgcd(e,phi)
        d = Integer(mod(bezout[1], phi))
        self.d = d
    def gen_key(self):
        return (self.e, self.d)
"""
class RSA_test:
    def __init__(self):
        p1 = 2147483647 # 2^31 - 1
        p2 = 2305843009213693951 # 2^61 - 1
        n = 4951760154835678088235319297 # p1*p2
        phi = 4951760152529835076874141700 # (p1-1)*(p2-1)
        
        self.e = 1850567623300615966303954877
        self.d = 4460824882019967172592779313
    def gen_key(self):
        return (self.e, self.d)
"""

'\nclass RSA_test:\n    def __init__(self):\n        p1 = 2147483647 # 2^31 - 1\n        p2 = 2305843009213693951 # 2^61 - 1\n        n = 4951760154835678088235319297 # p1*p2\n        phi = 4951760152529835076874141700 # (p1-1)*(p2-1)\n        \n        self.e = 1850567623300615966303954877\n        self.d = 4460824882019967172592779313\n    def gen_key(self):\n        return (self.e, self.d)\n'

In [19]:
# for simple case, let H(m) = m
def H(m):
    return m

In [20]:
# To realize DLP-based ch, I have to add a RSA encrypt (p,q)
# ref: Identity-Based Chameleon Hash and Applications
# G. Ateniese
class chameleon_hash:
    def __init__(self, Zq, q, N):
        self.Zq = Zq
        self.q = q
        self.N = N
    def Gen(self):
        rsa = RSA(self.N)
        e, t = rsa.gen_key()
        I =  self.Zq(randint(1, self.q))
        pk = [I, e]
        rho = self.Zq(randint(1, self.q))
        return (pk, rho, t)
    
    def CH(self, pk, m, rho):
        return pk[0]^H(m)*rho^pk[1]
    
    def UF(self, pk, t, m, rho, m_):
        B = pk[0]^t
        rho_ = rho*B^(H(m) - H(m_))
        return rho_

In [21]:
class signature_oracle:
    def __init__(self, g, q, N):
        self.g = g
        self.sk = randint(1, N)
    def Sign(self, m):
        return (m, self.g^self.sk)
    def Verify(self, m, sigma):
        return (sigma==self.Sign(m))

In [22]:
# has secret x
class Prover:
    def __init__(self, g, q, N):
        self.honest = True
        self.g = g
        self.q = q
        self.k = randint(1, q)
        self.x = -99999
        
    def Setup(self, S, p_set):
        r_ = self.g^self.k
        pk = p_set[0]
        CHF = p_set[1]
        rho = p_set[2]
        r = CHF.CH(pk, r_, rho)
        return (r_, r, S.Sign(r))
    
    def Prove(self, S, r, y):
        c = randint(1,self.q)
        s = self.k+c*self.x
        return (c, s, S.Sign(s))

In [23]:
class Verifier:
    def __init__(self, Zq, g, q, N):
        self.g = g
        self.t = -99999
        self.CHF = chameleon_hash(Zq, q, N)
        
    def KeyGen(self):
        return self.CHF.Gen()

    def Verify(self, S, pk, rho, y, r, c, s, sigma_r, sigma_s):
        v1 = S.Verify(r, sigma_r)
        v2 = S.Verify(s, sigma_s)
        rh_ = self.g^s * y^(-c)
        rh = self.CHF.CH(pk, rh_, rho)
        v3 = (r==rh)
        return (v1 and v2 and v3)

In [24]:
import time
class NTNIZKP_Protocol:
    def __init__(self, bits, seed):
        set_random_seed(seed)
        current_randstate().set_seed_gp()
        
        N = 2^bits
        q = random_prime(N)
        Zq = Integers(q)
        g = Zq(2)
        
        # knowledge
        x = randint(1, q)
        # such that
        y = g^x
        
        prover = Prover(g, q, N)
        verifier = Verifier(Zq, g, q, N)
        S = signature_oracle(g, q, N)
        
        if prover.honest:
            prover.x = x
        else:
            prover.x = randint(1, q)
        
        start = time.time()
        pk, rho, verifier.t = verifier.KeyGen()
        end = time.time()
        self.keygen_time = end - start
        
        p_set = [pk, verifier.CHF, rho]
        
        start = time.time()
        r_, r, sigma_r = prover.Setup(S, p_set)
        end = time.time()
        self.setup_time = end - start
        
        start = time.time()
        c, s, sigma_s = prover.Prove(S, r, y)
        end = time.time()
        self.prove_time = end-start
        
        start = time.time()
        self.result = verifier.Verify(S, pk, rho, y, r, c, s, sigma_r, sigma_s)
        end = time.time()
        self.verify_time = end - start

In [27]:
bits = 2048
ctr = 0
iteration = 20
total_keygen_time = 0
total_setup_time = 0
total_prove_time = 0
total_verify_time = 0

while ctr < iteration:
    seed = time.time_ns()
    print("iteration : %d / %d" % (ctr+1, iteration), end="\r")
    
    ntnizkp = NTNIZKP_Protocol(bits, seed)

    if ntnizkp.result:
        ctr += 1
        total_keygen_time += ntnizkp.keygen_time*10^6
        total_setup_time += ntnizkp.setup_time*10^6
        total_prove_time += ntnizkp.prove_time*10^6
        total_verify_time += ntnizkp.verify_time*10^6
    else:
        pass

print("\n-----------------------------")
print("Total keygen time / Average keygen time       : %.2fus / %.2fus" % (total_keygen_time, total_keygen_time/iteration))
print("Total setup time / Average setup time         : %.2fus / %.2fus" % (total_setup_time, total_setup_time/iteration))
print("Total prove time / Average prove time         : %.2fus / %.2fus" % (total_prove_time, total_prove_time/iteration))
print("Total verify time / Average verify time       : %.2fus / %.2fus" % (total_verify_time, total_verify_time/iteration))

iteration : 20 / 20
-----------------------------
Total keygen time / Average keygen time       : 1025163167.48ms / 51258158.37ms
Total setup time / Average setup time         : 396719.93ms / 19836.00ms
Total prove time / Average prove time         : 83060.74ms / 4153.04ms
Total verify time / Average verify time       : 647312.88ms / 32365.64ms
